In [1]:
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService
from cc3d.core.PyCoreSpecs import PottsCore, CellTypePlugin, VolumePlugin, ContactPlugin, BlobInitializer
from cc3d.core.PySteppables import SteppableBasePy

/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: PlayerSizes
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: PlayerSizesFloating
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')
/home/steve/miniconda3/envs/cc3d_env/lib/python3.10/site-packages/cc3d/core/Configuration/SettingUtils.py:248: UserWarning: Setting not set: RecentSimulations
  warnings.warn(f'Setting not set: {_el.attrib["Name"]}')


In [2]:
# Define the simulation lattice and neighborhood properties
# PottsCore sets up the basic simulation lattice dimensions and neighborhood order
potts_specs = PottsCore(dim_x=100, dim_y=100, neighbor_order=2)

# Define cell types for the simulation
# CellTypePlugin is used to specify different types of cells in the simulation
cell_type_specs = CellTypePlugin("Condensing", "NonCondensing")

# Define volume constraints for both cell types
# VolumePlugin sets the target volume and volume constraint strength for each cell type
volume_specs = VolumePlugin()
volume_specs.param_new("Condensing", target_volume=25, lambda_volume=2)
volume_specs.param_new("NonCondensing", target_volume=25, lambda_volume=2)

# Define adhesion (contact energy) between different cell types
# ContactPlugin sets the energy parameters for interactions between different cell types
contact_specs = ContactPlugin(neighbor_order=2)
contact_specs.param_new(type_1="Medium", type_2="Condensing", energy=20)
contact_specs.param_new(type_1="Medium", type_2="NonCondensing", energy=20)
contact_specs.param_new(type_1="Condensing", type_2="Condensing", energy=2)
contact_specs.param_new(type_1="Condensing", type_2="NonCondensing", energy=11)
contact_specs.param_new(type_1="NonCondensing", type_2="NonCondensing", energy=16)

# Initialize cells as a blob in the simulation space
# BlobInitializer sets up initial cell positions and types
blob_init_specs = BlobInitializer()
blob_init_specs.region_new(width=5, radius=20, center=(50, 50, 0), cell_types=("Condensing", "NonCondensing"))


In [3]:
# Define a custom steppable to implement cell growth logic
class GrowthSteppable(SteppableBasePy):
    def start(self):
        # Set initial volume parameters for each cell
        for cell in self.cell_list:
            cell.targetVolume = 25
            cell.lambdaVolume = 2

    def step(self, mcs):
        # Update target volume every 100 Monte Carlo Steps (MCS) to simulate growth
        if mcs % 100 == 0:
            for cell in self.cell_list:
                cell.targetVolume += 1  # Simple growth model

In [4]:
# Create a simulation service instance
cc3d_sim = CC3DSimService()

# Register all the specifications with the simulation service
cc3d_sim.register_specs([potts_specs, cell_type_specs, volume_specs, contact_specs, blob_init_specs])

# Register the custom steppable with the simulation service
cc3d_sim.register_steppable(steppable=GrowthSteppable, frequency=1)

In [5]:
# Run the simulation
cc3d_sim.run()

# Initialize the simulation
cc3d_sim.init()

# Start the simulation
cc3d_sim.start()

# Define the number of steps to run the simulation
num_steps = 1000

# Run the simulation for the specified number of steps
while cc3d_sim.current_step < num_steps:
    cc3d_sim.step()

Random number generator: MersenneTwister


WILL RUN SIMULATION FROM BEGINNING


In [6]:
import ipywidgets as widgets
from IPython.display import display
from cc3d.CompuCellSetup.CC3DCaller import CC3DSimService
from cc3d.core.PyCoreSpecs import PottsCore, CellTypePlugin, VolumePlugin, ContactPlugin, BlobInitializer
from cc3d.core.PySteppables import SteppableBasePy

# Create interactive widgets for user input
num_steps_widget = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Number of Steps:')
cell_type_dropdown = widgets.Dropdown(options=['Condensing', 'NonCondensing'], value='Condensing', description='Cell Type:')
target_volume_widget = widgets.IntSlider(value=25, min=1, max=100, step=1, description='Target Volume:')
lambda_volume_widget = widgets.FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Lambda Volume:')
energy_widget = widgets.IntSlider(value=16, min=1, max=30, step=1, description='Cell-Cell Energy:')

# Display the widgets
display(num_steps_widget, cell_type_dropdown, target_volume_widget, lambda_volume_widget, energy_widget)

# Define simulation lattice and neighborhood
potts_specs = PottsCore(dim_x=100, dim_y=100, neighbor_order=2)

# Define cell types
cell_type_specs = CellTypePlugin("Condensing", "NonCondensing")

# Volume constraint for both cell types
volume_specs = VolumePlugin()
volume_specs.param_new("Condensing", target_volume=target_volume_widget.value, lambda_volume=lambda_volume_widget.value)
volume_specs.param_new("NonCondensing", target_volume=target_volume_widget.value, lambda_volume=lambda_volume_widget.value)

# Adhesion (contact energy) between types
contact_specs = ContactPlugin(neighbor_order=2)
contact_specs.param_new(type_1="Medium", type_2="Condensing", energy=20)
contact_specs.param_new(type_1="Medium", type_2="NonCondensing", energy=20)
contact_specs.param_new(type_1="Condensing", type_2="Condensing", energy=2)
contact_specs.param_new(type_1="Condensing", type_2="NonCondensing", energy=11)
contact_specs.param_new(type_1="NonCondensing", type_2="NonCondensing", energy=energy_widget.value)

# Initialize cells as a blob
blob_init_specs = BlobInitializer()
blob_init_specs.region_new(width=5, radius=20, center=(50, 50, 0), cell_types=("Condensing", "NonCondensing"))

# Define a custom steppable to implement cell growth logic
class GrowthSteppable(SteppableBasePy):
    def start(self):
        # Set initial volume parameters for each cell
        for cell in self.cell_list:
            cell.targetVolume = target_volume_widget.value
            cell.lambdaVolume = lambda_volume_widget.value

    def step(self, mcs):
        # Update target volume every 100 Monte Carlo Steps (MCS) to simulate growth
        if mcs % 100 == 0:
            for cell in self.cell_list:
                cell.targetVolume += 1  # Simple growth model

# Create a simulation service instance
cc3d_sim = CC3DSimService()

# Register all the specifications with the simulation service
cc3d_sim.register_specs([potts_specs, cell_type_specs, volume_specs, contact_specs, blob_init_specs])

# Register the custom steppable with the simulation service
cc3d_sim.register_steppable(steppable=GrowthSteppable, frequency=1)

# Run the simulation
cc3d_sim.run()
cc3d_sim.init()
cc3d_sim.start()

# Run the simulation for the specified number of steps
while cc3d_sim.current_step < num_steps_widget.value:
    cc3d_sim.step()


IntSlider(value=1000, description='Number of Steps:', max=10000, min=100, step=100)

Dropdown(description='Cell Type:', options=('Condensing', 'NonCondensing'), value='Condensing')

IntSlider(value=25, description='Target Volume:', min=1)

FloatSlider(value=2.0, description='Lambda Volume:', max=10.0, min=0.1)

IntSlider(value=16, description='Cell-Cell Energy:', max=30, min=1)

Random number generator: MersenneTwister


WILL RUN SIMULATION FROM BEGINNING
